In [1]:
from db_scripts import *
from utils import *
from constant import *
from config import *

# Chemin de sauvegarde des données à télécharger
data_path = "data"

In [3]:
# Création de la base de données
create_database("censo_escolar")

Database 'censo_escolar' already exists.


In [ ]:
# Téléchargement des données
folderName  = data_path + "/csv"
years = [str(i) for i in range(2020, 2022+1)]


download_csv(folderName, years)

In [2]:
# Création de la session spark
spark = create_spark_session()

24/05/22 16:55:04 WARN Utils: Your hostname, Asma resolves to a loopback address: 127.0.1.1; using 192.168.1.21 instead (on interface wlp0s20f3)
24/05/22 16:55:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/05/22 16:55:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Spark session created successfully


In [3]:
# Chargement des fichiers csv avec spark
data_cvs = read_csv(spark)

In [7]:
display(data_cvs.head(2))


24/05/22 16:21:18 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


[Row(NU_ANO_CENSO='2020', NO_REGIAO='Norte', CO_REGIAO='1', NO_UF='Rondônia', SG_UF='RO', CO_UF='11', NO_MUNICIPIO="Alta Floresta D'Oeste", CO_MUNICIPIO='1100015', NO_MESORREGIAO='Leste Rondoniense', CO_MESORREGIAO='1102', NO_MICRORREGIAO='Cacoal', CO_MICRORREGIAO='11006', CO_DISTRITO='110001505', CO_ENTIDADE='11022558', NO_ENTIDADE='EIEEF HAP BITT TUPARI', TP_DEPENDENCIA='2', TP_CATEGORIA_ESCOLA_PRIVADA='0', TP_LOCALIZACAO='2', TP_LOCALIZACAO_DIFERENCIADA='2', DS_ENDERECO='TERRA INDIGENA RIO BRANCO', NU_ENDERECO=None, DS_COMPLEMENTO='ALDEIA COLORADO', NO_BAIRRO='RURAL', CO_CEP='76954000', NU_DDD='69', NU_TELEFONE='36413839', TP_SITUACAO_FUNCIONAMENTO='1', CO_ORGAO_REGIONAL='00015', DT_ANO_LETIVO_INICIO='06FEB2020:00:00:00', DT_ANO_LETIVO_TERMINO='11DEC2020:00:00:00', IN_VINCULO_SECRETARIA_EDUCACAO='1', IN_VINCULO_SEGURANCA_PUBLICA='0', IN_VINCULO_SECRETARIA_SAUDE='0', IN_VINCULO_OUTRO_ORGAO='0', IN_CONVENIADA_PP='0', TP_CONVENIO_PODER_PUBLICO='0', IN_MANT_ESCOLA_PRIVADA_EMP='0', IN_MA

In [4]:
# Convertion des fichiers csv en parquets
folderPath = data_path + '/parquets'
convert_to_parquet(data_cvs, folderPath)

24/05/22 16:27:43 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
24/05/22 16:27:45 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 385, schema size: 370
CSV file: file:///home/mohamed/projects/data_pipline_spark/data/csv/2022.csv
24/05/22 16:27:45 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [4]:
# Chargement des fichiers parquets avec spark
data_parquet = read_parquet(spark)

In [6]:
display(data_parquet.head(2))

[Row(NU_ANO_CENSO='2022', NO_REGIAO='Nordeste', CO_REGIAO='2', NO_UF='Bahia', SG_UF='BA', CO_UF='29', NO_MUNICIPIO='Jaborandi', CO_MUNICIPIO='2917359', NO_MESORREGIAO='Extremo Oeste Baiano', CO_MESORREGIAO='1', NO_MICRORREGIAO='Santa Maria da Vitória', CO_MICRORREGIAO='3', CO_DISTRITO='291735905', CO_ENTIDADE='ESC MUN JOAQUIM CANDIDO RODRIGUES', NO_ENTIDADE='29386136', TP_DEPENDENCIA='3', TP_CATEGORIA_ESCOLA_PRIVADA=None, TP_LOCALIZACAO='1', TP_LOCALIZACAO_DIFERENCIADA='0', DS_ENDERECO='RUA DOM MUNIZ', NU_ENDERECO='SN', DS_COMPLEMENTO='PREDIO ESCOLAR', NO_BAIRRO='CENTRO', CO_CEP='47655000', NU_DDD='77', NU_TELEFONE='36832152', TP_SITUACAO_FUNCIONAMENTO='1', CO_ORGAO_REGIONAL='00023', DT_ANO_LETIVO_INICIO='07FEB2022:00:00:00', DT_ANO_LETIVO_TERMINO='19DEC2022:00:00:00', IN_VINCULO_SECRETARIA_EDUCACAO='1', IN_VINCULO_SEGURANCA_PUBLICA='0', IN_VINCULO_SECRETARIA_SAUDE='0', IN_VINCULO_OUTRO_ORGAO='0', IN_CONVENIADA_PP='0', TP_CONVENIO_PODER_PUBLICO=None, IN_MANT_ESCOLA_PRIVADA_EMP=None, IN

In [5]:
data_parquet\
.select(
    # Selecting the dimension table's columns
    [F.col(field).cast("string") for field in DIM_COLUMNS]
)\
.distinct()\
.withColumn( # Adding an distinct id for each unque record 
    "id", F.monotonically_increasing_id()
)\
.write\
.jdbc(
    url = POSTGRES_URL,
    table="DIM_LOCAL",
    mode="overwrite",
    properties = POSTGRES_PROPERTIES,
)

24/05/22 16:55:50 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [6]:
# Connexion à la base de données censor escolar
connection = connect_to_db("censo_escolar")
cursor = connection.cursor()

cursor.execute(
    "ALTER TABLE DIM_LOCAL ADD PRIMARY KEY (id);"
)
cursor.close()
connection.commit()

In [7]:
from datetime import datetime
from config import DIMENSION_TABLES_CONFIG


for table_name, table_config in DIMENSION_TABLES_CONFIG.items():
    
    print(f"[{datetime.now()}] Writing {table_name}")
    
    data_parquet\
    .select(
        [
            F
            .col(field["field"])
            .cast(field["type"])
            .alias(field["field"])
            
            for field
            in table_config["fields"]
        ]
    )\
    .distinct()\
    .withColumn(
        "id", F.monotonically_increasing_id()
    )\
    .write\
    .jdbc(
        url = POSTGRES_URL,
        table=table_name,
        mode="overwrite",
        properties = POSTGRES_PROPERTIES,
    )
    
    print(f"[{datetime.now()}] Wrote {table_name}")
    # Define id as the primary key
    cursor = connection.cursor()
    cursor.execute(
        f"ALTER TABLE {table_name} ADD PRIMARY KEY (id);"
    )
    cursor.close()
    connection.commit()

    print(f"[{datetime.now()}] Added primary key to {table_name}")
    print(f"[{datetime.now()}] Done")

[2024-05-22 16:56:12.413147] Writing DIM_LOCAL


[2024-05-22 16:56:13.828283] Wrote DIM_LOCAL
[2024-05-22 16:56:13.843618] Added primary key to DIM_LOCAL
[2024-05-22 16:56:13.843682] Done
[2024-05-22 16:56:13.843697] Writing DIM_TP_DEPENDENCIA
[2024-05-22 16:56:14.979554] Wrote DIM_TP_DEPENDENCIA
[2024-05-22 16:56:14.985027] Added primary key to DIM_TP_DEPENDENCIA
[2024-05-22 16:56:14.985098] Done
[2024-05-22 16:56:14.985114] Writing DIM_TP_LOCALIZACAO
[2024-05-22 16:56:15.572564] Wrote DIM_TP_LOCALIZACAO
[2024-05-22 16:56:15.577852] Added primary key to DIM_TP_LOCALIZACAO
[2024-05-22 16:56:15.577912] Done
[2024-05-22 16:56:15.577932] Writing DIM_IN_AGUA_POTAVEL
[2024-05-22 16:56:16.269389] Wrote DIM_IN_AGUA_POTAVEL
[2024-05-22 16:56:16.274997] Added primary key to DIM_IN_AGUA_POTAVEL
[2024-05-22 16:56:16.275064] Done
[2024-05-22 16:56:16.275079] Writing DIM_IN_ENERGIA_INEXISTENTE
[2024-05-22 16:56:16.840233] Wrote DIM_IN_ENERGIA_INEXISTENTE
[2024-05-22 16:56:16.845318] Added primary key to DIM_IN_ENERGIA_INEXISTENTE
[2024-05-22 16:5

In [35]:
# Create fact table
# Using the configuration in FACT_CONFIG
# With id as the primary key

# Avoid inserting a backslash into a f-string


comma_break_line = ",\n\t\t\t"

facts_table_sql = f"""

    CREATE TABLE IF NOT EXISTS {FACT_TABLE_NAME} (
        id SERIAL PRIMARY KEY,
        { 
            comma_break_line.join(
                [
                    f"{field} VARCHAR" 
                    for field in FACT_COLUMNS
                ]
                +[
                    f"ID_{dim_table} BIGINT"
                    for dim_table in DIMENSION_ID_CONFIG.keys()
                ]
            )
        }
    );
    
    -- Adding Foreign Keys
    ALTER TABLE {FACT_TABLE_NAME}
    {
        comma_break_line.join(
            [
                f"ADD CONSTRAINT {FACT_TABLE_NAME}_{dim_table}_fk FOREIGN KEY (ID_{dim_table}) REFERENCES {dim_table}(id)"
                for dim_table in DIMENSION_ID_CONFIG.keys()
            ]
        )
    }
"""

In [36]:
# Execution de la requête de creation de la table de faits
print(f"[{datetime.now()}] Creating facts table")
cursor = connection.cursor()
try:
    cursor.execute(facts_table_sql)
    cursor.close()
    connection.commit()
except Exception as e:
    print(e)
    connection.rollback()
    cursor.close()
else:
    print(f"[{datetime.now()}] Created facts table")
    print(f"[{datetime.now()}] Done")

[2024-05-22 18:23:35.769214] Creating facts table
[2024-05-22 18:23:35.792975] Created facts table
[2024-05-22 18:23:35.793097] Done


In [37]:
# Récupération des colonnes qui seront utilisées
from itertools import chain


facts_data = data_parquet\
    .select(
        [
            *chain(
                *DIMENSION_ID_CONFIG.values(), 
                FACT_CONFIG.keys()
            )
        ]
    )

In [38]:
# Joining the id of the dimensions

for table_name, table_fields in DIMENSION_ID_CONFIG.items():
    
    # Read the dimension data from Postgres
    dim_table = spark.read\
         .jdbc(
            url = POSTGRES_URL,
            table=table_name,
            properties = POSTGRES_PROPERTIES,
        )\
        .withColumnRenamed("id", f"ID_{table_name}")
    
    # Join the dimension data with the fact data
    facts_data = facts_data\
        .join(
            dim_table,
            on=table_fields,
            how="left"
        )\
        .drop(*table_fields)

In [39]:
# Order the columns to match the fact table on postgres
# and save the data
facts_data\
    .select(*FACT_TABLE_ALL_COLUMNS_ORDERED)\
    .write\
    .jdbc(
        url = POSTGRES_URL,
        table=FACT_TABLE_NAME,
        mode="append",
        properties = POSTGRES_PROPERTIES,
    )